In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Target:

    A basic working skeleton for digit classification using MNIST dataset
    Reduced number of channels in each layer for the training to be faster

Result:

    Parameters : 49,450
    Epochs     : 20  
    Train accuracy = 99.60%
    Test accuracy = 99.17%

Analysis:

    Since test accuracy is slightly less than train accuracy - there is small amount of overfitting
    The test accuracy is fluctuating between 98% to 99.17%
    


In [0]:
# Original network with less parameters, so that training is also faster
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop2 = nn.Dropout(0.1)  
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bnm2d1 = nn.BatchNorm2d(32) 
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.bnm2d2 = nn.BatchNorm2d(32) 
        self.conv5 = nn.Conv2d(32, 32, 3)
        self.conv6 = nn.Conv2d(32, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
            Conv2d-5           [-1, 32, 14, 14]           9,248
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           9,248
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 49,450
Trainable params: 49,450
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.55
Params size (MB): 0.19
Estimated Total Size (MB): 0.74
---------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
train_acc = []
test_acc = []
train_losses = []
test_losses = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        y_pred = model(data)

        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
    # def __init__(self):
    #     super(Net, self).__init__()
    #     self.drop2 = nn.Dropout(0.1)  
    #     self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
    #     self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.pool1 = nn.MaxPool2d(2, 2)
    #     self.bnm2d1 = nn.BatchNorm2d(32) 
    #     self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.pool2 = nn.MaxPool2d(2, 2)
    #     self.bnm2d2 = nn.BatchNorm2d(32) 
    #     self.conv5 = nn.Conv2d(32, 32, 3)
    #     self.conv6 = nn.Conv2d(32, 32, 3)
    #     self.conv7 = nn.Conv2d(32, 10, 3)

    # def forward(self, x):
    #     x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    #     x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    #     x = F.relu(self.conv6(F.relu(self.conv5(x))))
    #     x = self.conv7(x)
    #     x = x.view(-1, 10)
    #     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.07452227920293808 batch_id=468 Accuracy=76.50: 100%|██████████| 469/469 [00:12<00:00, 37.62it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0908, Accuracy: 9710/10000 (97.10%)



loss=0.03752938285470009 batch_id=468 Accuracy=97.20: 100%|██████████| 469/469 [00:12<00:00, 37.93it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0646, Accuracy: 9800/10000 (98.00%)



loss=0.041007548570632935 batch_id=468 Accuracy=98.07: 100%|██████████| 469/469 [00:12<00:00, 38.41it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0282, Accuracy: 9910/10000 (99.10%)



loss=0.02394961379468441 batch_id=468 Accuracy=99.66: 100%|██████████| 469/469 [00:11<00:00, 40.70it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0286, Accuracy: 9917/10000 (99.17%)



loss=0.0005345146055333316 batch_id=468 Accuracy=99.70: 100%|██████████| 469/469 [00:11<00:00, 40.27it/s]



Test set: Average loss: 0.0381, Accuracy: 9891/10000 (98.91%)

